# **Notebook 3: Feature Engineering**

## Objectives

* Generate train and test sets for feature engineering.
* Engineer features for Classification, Regression and Cluster models

## Inputs

* outputs/datasets/cleaned/house_prices_records_cleaned.csv

## Outputs

* Generate Train and Test sets from cleaned data, saved under outputs/datasets/cleaned/train and outputs/datasets/cleaned/test

## Conclusions

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/housing/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/housing'

# Load data

In [4]:
import pandas as pd
df_raw_path = "outputs/datasets/cleaned/house_prices_records_cleaned.csv"
df = pd.read_csv(df_raw_path)
df.head(3)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GarageYrBlt,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854,3,No,706,GLQ,150,548,RFn,2003,...,8450,65,196,61,5,7,856,2003,2003,208500
1,1262,0,3,Gd,978,ALQ,284,460,RFn,1976,...,9600,80,0,0,8,6,1262,1976,1976,181500
2,920,866,3,Mn,486,GLQ,434,608,RFn,2001,...,11250,68,162,42,5,7,920,2001,2002,223500


# Split train and test data

In the cell below, the target variable 'SalePrice' is separated out from the rest of the data, and the split produces both a train and test set for the features (TrainSet and TestSet) and the target (train_target and test_target).

The reason for changing the code from the walkthrough is that in the train_test_split() function, I was passing df(['SalePrice']) as the target variable, but also including it in my df, which means my target variable 'SalePrice' would be present in both my features and targets, which is not what I would typically want.

In [22]:
features = df.drop('SalePrice', axis=1)  # drop the target variable from the feature set
target = df['SalePrice']

TrainSet, TestSet, train_target, test_target = train_test_split(
    features,
    target,
    test_size=0.2,
    random_state=0
)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")


TrainSet shape: (1168, 21) 
TestSet shape: (292, 21)


As we see in the output, the train set has 1168 rows which is 80% of the data, and the test set accounts for the remaining 20%.

We then save the train and test set respectively in their folders.

In [23]:
TrainSet.to_csv("outputs/datasets/cleaned/train/TrainSetCleaned.csv", index=False)
TestSet.to_csv("outputs/datasets/cleaned/test/TestSetCleaned.csv", index=False)

And we also save the datasets where we put the target variable.

In [18]:
train_target.to_csv("outputs/datasets/cleaned/train/TrainSetTarget.csv", index=False)
test_target.to_csv("outputs/datasets/cleaned/test/TestSetTarget.csv", index=False)

# Correlation and PPS Analysis

* Since we did data cleaning before the EDA and analyses, we don’t expect any changes in correlation levels and PPS compared to the previous notebook.

# FEATURE ENGINEERING

Now we move on to Feature Engineering.

In this section we will handle outliers, encode categorical variables, transform features and create new features. 


# Load cleaned training and test sets

In [12]:
import pandas as pd

TrainSet_df = pd.read_csv("outputs/datasets/cleaned/train/TrainSetCleaned.csv")
TestSet_df = pd.read_csv("outputs/datasets/cleaned/test/TestSetCleaned.csv")

# Check the first few rows of the DataFrame to confirm that it loaded correctly
TrainSet_df.head()
TestSet_df.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GarageYrBlt,...,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd
0,2515,0,4,No,1219,Rec,816,484,Unf,1975,...,TA,32668,69,0,0,3,6,2035,1957,1975
1,958,620,3,No,403,BLQ,238,240,Unf,1941,...,Fa,9490,79,0,0,7,6,806,1941,1950
2,979,224,3,No,185,LwQ,524,352,Unf,1950,...,Gd,7015,69,161,0,4,5,709,1950,1950
3,1156,866,4,No,392,BLQ,768,505,Fin,1977,...,TA,10005,83,299,117,5,7,1160,1977,1977
4,525,0,3,No,0,Unf,525,264,Unf,1971,...,TA,1680,21,381,0,5,6,525,1971,1971


First we look at encoding our categorical variables since many machine learning models require inputs to be numerical. Luckily, most of our variables are numerical already, but we do have 4 categorical variables that need to be converted into numerical form.

We will use Ordinal Encoding. This method converts each category to a unique integer. It is suitable for ordinal variables, where there is an inherent order in the categories.

Looking at our categorical variables:

**BsmtExposure**, **BsmtFinType1**, **GarageFinish** seem to be ordinal variables as they indicate quality or condition, with 'None' likely representing a missing or not applicable condition.

**KitchenQual** seems to be an ordinal variable as well, with an inherent order (Ex > Gd > TA > Fa). 

Great!

# Categorical encoding

In [14]:
from sklearn.preprocessing import OrdinalEncoder

# Define a dictionary with the correct order of each category for each variable
categories_dict = {
    'BsmtExposure': ['None', 'No', 'Mn', 'Av', 'Gd'],
    'BsmtFinType1': ['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'GarageFinish': ['None', 'Unf', 'RFn', 'Fin'],
    'KitchenQual': ['Fa', 'TA', 'Gd', 'Ex'],
}

# Convert dict_keys object to a list
categories_list = list(categories_dict.keys())

# Instantiate the encoder
encoder = OrdinalEncoder(categories=list(categories_dict.values()))

# Fit the encoder on the categorical columns of the training data
encoder.fit(TrainSet_df[categories_dict.keys()])

# Apply the encoder to the training data
TrainSet_df_encoded = pd.DataFrame(encoder.transform(TrainSet_df[categories_dict.keys()]), columns=categories_dict.keys())

# Apply the encoder to the test data
TestSet_df_encoded = pd.DataFrame(encoder.transform(TestSet_df[categories_dict.keys()]), columns=categories_dict.keys())

# Replace the old categorical columns in the dataframes with the new encoded ones
TrainSet_df[categories_dict.keys()] = TrainSet_df_encoded
TestSet_df[categories_dict.keys()] = TestSet_df_encoded


TypeError: unhashable type: 'dict_keys'

# Feature Engineering Analysis

In [5]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
from feature_engine import transformation as vt
from feature_engine.outliers import Winsorizer
from feature_engine.encoding import OrdinalEncoder
sns.set(style="whitegrid")
warnings.filterwarnings('ignore')


def FeatureEngineeringAnalysis(df, analysis_type='numerical'):
    """
    - used for quick feature engineering on numerical and categorical variables
    to decide which transformation can better transform the distribution shape
    - Once transformed, use a reporting tool, like pandas-profiling, to evaluate distributions
    """
    check_missing_values(df)
    allowed_types = ['numerical', 'ordinal_encoder', 'outlier_winsorizer']
    check_user_entry_on_analysis_type(analysis_type, allowed_types)
    list_column_transformers = define_list_column_transformers(analysis_type)

    # Loop in each variable and engineer the data according to the analysis type
    df_feat_eng = pd.DataFrame([])
    for column in df.columns:
        # create additional columns (column_method) to apply the methods
        df_feat_eng = pd.concat([df_feat_eng, df[column]], axis=1)
        for method in list_column_transformers:
            df_feat_eng[f"{column}_{method}"] = df[column]

        # Apply transformers in respective column_transformers
        df_feat_eng, list_applied_transformers = apply_transformers(
            analysis_type, df_feat_eng, column)

        # For each variable, assess how the transformations perform
        transformer_evaluation(
            column, list_applied_transformers, analysis_type, df_feat_eng)

    return df_feat_eng


def check_user_entry_on_analysis_type(analysis_type, allowed_types):
    """ Check analysis type """
    if analysis_type is None:
        raise SystemExit(
            f"You should pass analysis_type parameter as one of the following options: {allowed_types}")
    if analysis_type not in allowed_types:
        raise SystemExit(
            f"analysis_type argument should be one of these options: {allowed_types}")


def check_missing_values(df):
    if df.isna().sum().sum() != 0:
        raise SystemExit(
            f"There is a missing value in your dataset. Please handle that before getting into feature engineering.")


def define_list_column_transformers(analysis_type):
    """ Set suffix columns according to analysis_type"""
    if analysis_type == 'numerical':
        list_column_transformers = [
            "log_e", "log_10", "reciprocal", "power", "box_cox", "yeo_johnson"]

    elif analysis_type == 'ordinal_encoder':
        list_column_transformers = ["ordinal_encoder"]

    elif analysis_type == 'outlier_winsorizer':
        list_column_transformers = ['iqr']

    return list_column_transformers


def apply_transformers(analysis_type, df_feat_eng, column):
    for col in df_feat_eng.select_dtypes(include='category').columns:
        df_feat_eng[col] = df_feat_eng[col].astype('object')

    if analysis_type == 'numerical':
        df_feat_eng, list_applied_transformers = FeatEngineering_Numerical(
            df_feat_eng, column)

    elif analysis_type == 'outlier_winsorizer':
        df_feat_eng, list_applied_transformers = FeatEngineering_OutlierWinsorizer(
            df_feat_eng, column)

    elif analysis_type == 'ordinal_encoder':
        df_feat_eng, list_applied_transformers = FeatEngineering_CategoricalEncoder(
            df_feat_eng, column)

    return df_feat_eng, list_applied_transformers


def transformer_evaluation(column, list_applied_transformers, analysis_type, df_feat_eng):
    # For each variable, assess how the transformations perform
    print(f"* Variable Analyzed: {column}")
    print(f"* Applied transformation: {list_applied_transformers} \n")
    for col in [column] + list_applied_transformers:

        if analysis_type != 'ordinal_encoder':
            DiagnosticPlots_Numerical(df_feat_eng, col)

        else:
            if col == column:
                DiagnosticPlots_Categories(df_feat_eng, col)
            else:
                DiagnosticPlots_Numerical(df_feat_eng, col)

        print("\n")


def DiagnosticPlots_Categories(df_feat_eng, col):
    plt.figure(figsize=(4, 3))
    sns.countplot(data=df_feat_eng, x=col, palette=[
                  '#432371'], order=df_feat_eng[col].value_counts().index)
    plt.xticks(rotation=90)
    plt.suptitle(f"{col}", fontsize=30, y=1.05)
    plt.show()
    print("\n")


def DiagnosticPlots_Numerical(df, variable):
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    sns.histplot(data=df, x=variable, kde=True, element="step", ax=axes[0])
    stats.probplot(df[variable], dist="norm", plot=axes[1])
    sns.boxplot(x=df[variable], ax=axes[2])

    axes[0].set_title('Histogram')
    axes[1].set_title('QQ Plot')
    axes[2].set_title('Boxplot')
    fig.suptitle(f"{variable}", fontsize=30, y=1.05)
    plt.tight_layout()
    plt.show()


def FeatEngineering_CategoricalEncoder(df_feat_eng, column):
    list_methods_worked = []
    try:
        encoder = OrdinalEncoder(encoding_method='arbitrary', variables=[
                                 f"{column}_ordinal_encoder"])
        df_feat_eng = encoder.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_ordinal_encoder")

    except Exception:
        df_feat_eng.drop([f"{column}_ordinal_encoder"], axis=1, inplace=True)

    return df_feat_eng, list_methods_worked


def FeatEngineering_OutlierWinsorizer(df_feat_eng, column):
    list_methods_worked = []

    # Winsorizer iqr
    try:
        disc = Winsorizer(
            capping_method='iqr', tail='both', fold=1.5, variables=[f"{column}_iqr"])
        df_feat_eng = disc.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_iqr")
    except Exception:
        df_feat_eng.drop([f"{column}_iqr"], axis=1, inplace=True)

    return df_feat_eng, list_methods_worked


def FeatEngineering_Numerical(df_feat_eng, column):
    list_methods_worked = []

    # LogTransformer base e
    try:
        lt = vt.LogTransformer(variables=[f"{column}_log_e"])
        df_feat_eng = lt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_log_e")
    except Exception:
        df_feat_eng.drop([f"{column}_log_e"], axis=1, inplace=True)

    # LogTransformer base 10
    try:
        lt = vt.LogTransformer(variables=[f"{column}_log_10"], base='10')
        df_feat_eng = lt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_log_10")
    except Exception:
        df_feat_eng.drop([f"{column}_log_10"], axis=1, inplace=True)

    # ReciprocalTransformer
    try:
        rt = vt.ReciprocalTransformer(variables=[f"{column}_reciprocal"])
        df_feat_eng = rt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_reciprocal")
    except Exception:
        df_feat_eng.drop([f"{column}_reciprocal"], axis=1, inplace=True)

    # PowerTransformer
    try:
        pt = vt.PowerTransformer(variables=[f"{column}_power"])
        df_feat_eng = pt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_power")
    except Exception:
        df_feat_eng.drop([f"{column}_power"], axis=1, inplace=True)

    # BoxCoxTransformer
    try:
        bct = vt.BoxCoxTransformer(variables=[f"{column}_box_cox"])
        df_feat_eng = bct.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_box_cox")
    except Exception:
        df_feat_eng.drop([f"{column}_box_cox"], axis=1, inplace=True)

    # YeoJohnsonTransformer
    try:
        yjt = vt.YeoJohnsonTransformer(variables=[f"{column}_yeo_johnson"])
        df_feat_eng = yjt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_yeo_johnson")
    except Exception:
        df_feat_eng.drop([f"{column}_yeo_johnson"], axis=1, inplace=True)

    return df_feat_eng, list_methods_worked

In [6]:
df_transformed = FeatureEngineeringAnalysis(df, analysis_type='numerical')  # replace 'numerical' with the analysis type you want


NameError: name 'df' is not defined

# Next step

* Great! Now you can  push the changes to your GitHub Repo, using the Git commands (git add, git commit, git push)